In [1]:
import os
import time
import threading
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
from webdriver_manager.chrome import ChromeDriverManager

In [3]:
# 설정
SEARCH_KEYWORD = "문상훈"
NUMBER_OF_IMAGES = 20000
THREAD_COUNT = 3
SAVE_DIR = "pinterest_images"

# 검색 변형 리스트
search_variants = [
    SEARCH_KEYWORD,
    f"{SEARCH_KEYWORD} 밈",
    f"{SEARCH_KEYWORD} 짤"
]

# 저장 폴더 생성
if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)

def download_image(url, save_path, retries=3):
    while retries > 0:
        try:
            response = requests.get(url, stream=True, timeout=10)
            if response.status_code == 200:
                with open(save_path, 'wb') as file:
                    for chunk in response.iter_content(1024):
                        file.write(chunk)
                return True
            else:
                retries -= 1
        except Exception:
            retries -= 1
    return False

def scroll_and_collect_images(search_query, thread_index, progress):
    driver = None
    try:
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument("--disable-gpu")
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
        
        # 로그인 및 검색
        driver.get(f"https://kr.pinterest.com/search/pins/?q={search_query}&rs=typed")

        image_urls = set()
        scroll_pause_time = 1
        retries = 5
        wait = WebDriverWait(driver, 10)
        scroll_attempts = 0
        max_scroll_attempts = 50

        while len(image_urls) < NUMBER_OF_IMAGES // THREAD_COUNT:
            try:
                driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
                time.sleep(scroll_pause_time)
                scroll_attempts += 1
                
                image_elements = wait.until(EC.presence_of_all_elements_located((By.TAG_NAME, "img")))
                for img_element in image_elements:
                    img_url = img_element.get_attribute("src")
                    if img_url and img_url.startswith("https") and img_url not in image_urls:
                        image_urls.add(img_url)
                        if len(image_urls) >= NUMBER_OF_IMAGES // THREAD_COUNT:
                            break

                if scroll_attempts >= max_scroll_attempts:
                    print(f"Thread {thread_index}: 스크롤 한계에 도달하여 크롤링을 종료합니다.")
                    break
            except StaleElementReferenceException:
                continue
            except Exception as e:
                retries -= 1
                if retries == 0:
                    print(f"Failed to scroll in thread {thread_index} after multiple retries: {e}")
                    break
                else:
                    time.sleep(1)
        
        # 이미지 저장
        for idx, img_url in enumerate(image_urls):
            file_name = f"{search_query.replace(' ', '_')}_{thread_index}_{idx}.jpg"
            save_path = os.path.join(SAVE_DIR, file_name)
            if download_image(img_url, save_path):
                progress[thread_index] += 1
                total_downloaded = sum(progress)
                if total_downloaded % 100 == 0:
                    print(f"Total: {total_downloaded}/{NUMBER_OF_IMAGES} images downloaded.")
                    print(f"Progress: {total_downloaded / NUMBER_OF_IMAGES * 100:.2f}% completed.")
    finally:
        if driver is not None:
            driver.quit()

def start_crawling():
    threads = []
    progress = [0] * THREAD_COUNT

    for search_variant in search_variants:
        for i in range(THREAD_COUNT):
            thread = threading.Thread(
                target=scroll_and_collect_images,
                args=(search_variant, i, progress)
            )
            threads.append(thread)
            thread.start()
    
    for thread in threads:
        thread.join()
    
    print(f"크롤링이 완료되었습니다. 이미지는 '{SAVE_DIR}' 폴더에 저장되었습니다.")

if __name__ == "__main__":
    try:
        start_crawling()
    except ModuleNotFoundError as e:
        print("Module 'webdriver_manager' not found. Please install it using 'pip install webdriver-manager'.")
    except Exception as e:
        print(f"An error occurred: {e}")

Thread 1: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Thread 3: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Thread 0: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Thread 2: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Thread 1: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Thread 4: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Thread 0: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Thread 2: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Thread 4: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Thread 3: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Total: 100/20000 images downloaded.
Progress: 0.50% completed.
Thread 1: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Total: 200/20000 images downloaded.
Progress: 1.00% completed.
Thread 3: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Total: 300/20000 images downloaded.
Progress: 1.50% completed.
Thread 2: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Thread 0: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Total: 400/20000 images downloaded.
Progress: 2.00% completed.
Thread 4: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Total: 500/20000 images downloaded.
Progress: 2.50% completed.
Total: 600/20000 images downloaded.
Progress: 3.00% completed.
Total: 700/20000 images downloaded.
Progress: 3.50% completed.
Total: 800/20000 images downloaded

In [7]:
# 검색어 목록
search_keywords = [
    "루피", "절망", "김용명", "소주", "기쁨", "슬픔", "화남", "치킨", "커비", "감자"
]

# 이후 코드에서 search_keywords를 사용하여 모든 검색어에 대해 자동 크롤링을 수행

"""
"고양이", "강아지", "잠", "사랑해", "티발롬", "자퇴", "농담곰", "햄스터", "눈물",
    "짱구", "기프티콘", "왹져"
"""

'\n"고양이", "강아지", "잠", "사랑해", "티발롬", "자퇴", "농담곰", "햄스터", "눈물",\n    "짱구", "기프티콘", "왹져"\n'

In [11]:
# 설정
NUMBER_OF_IMAGES = 20000
THREAD_COUNT = 5

In [13]:
# 검색 변형 리스트 생성 함수
def get_search_variants(keyword):
    return [
        keyword,
        f"{keyword} 밈",
        f"{keyword} 짤"
    ]

def download_image(url, save_path, retries=3):
    while retries > 0:
        try:
            response = requests.get(url, stream=True, timeout=10)
            if response.status_code == 200:
                with open(save_path, 'wb') as file:
                    for chunk in response.iter_content(1024):
                        file.write(chunk)
                return True
            else:
                retries -= 1
        except Exception:
            retries -= 1
    return False

def scroll_and_collect_images(search_query, thread_index, save_dir, progress):
    driver = None
    try:
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument("--disable-gpu")
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
        
        # 검색 실행
        driver.get(f"https://kr.pinterest.com/search/pins/?q={search_query}&rs=typed")

        image_urls = set()
        scroll_pause_time = 1
        retries = 5
        wait = WebDriverWait(driver, 10)
        scroll_attempts = 0
        max_scroll_attempts = 50

        while len(image_urls) < NUMBER_OF_IMAGES // THREAD_COUNT:
            try:
                driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
                time.sleep(scroll_pause_time)
                scroll_attempts += 1
                
                image_elements = wait.until(EC.presence_of_all_elements_located((By.TAG_NAME, "img")))
                for img_element in image_elements:
                    img_url = img_element.get_attribute("src")
                    if img_url and img_url.startswith("https") and img_url not in image_urls:
                        image_urls.add(img_url)
                        if len(image_urls) >= NUMBER_OF_IMAGES // THREAD_COUNT:
                            break

                if scroll_attempts >= max_scroll_attempts:
                    print(f"Thread {thread_index}: 스크롤 한계에 도달하여 크롤링을 종료합니다.")
                    break
            except StaleElementReferenceException:
                continue
            except Exception as e:
                retries -= 1
                if retries == 0:
                    print(f"Failed to scroll in thread {thread_index} after multiple retries: {e}")
                    break
                else:
                    time.sleep(1)
        
        # 이미지 저장
        for idx, img_url in enumerate(image_urls):
            file_name = f"{search_query.replace(' ', '_')}_{thread_index}_{idx}.jpg"
            save_path = os.path.join(save_dir, file_name)
            if download_image(img_url, save_path):
                progress[thread_index] += 1
                total_downloaded = sum(progress)
                if total_downloaded % 100 == 0:
                    print(f"Total: {total_downloaded}/{NUMBER_OF_IMAGES} images downloaded.")
                    print(f"Progress: {total_downloaded / NUMBER_OF_IMAGES * 100:.2f}% completed.")
    finally:
        if driver is not None:
            driver.quit()

def start_crawling():
    for keyword in search_keywords:
        print(f"Starting crawl for keyword: {keyword}")
        search_variants = get_search_variants(keyword)
        
        # 각 검색어별 폴더 생성
        keyword_dir = os.path.join(SAVE_DIR, keyword)
        if not os.path.exists(keyword_dir):
            os.makedirs(keyword_dir)
        
        for search_variant in search_variants:
            print(f"Processing search variant: {search_variant}")
            threads = []
            progress = [0] * THREAD_COUNT
            
            for i in range(THREAD_COUNT):
                thread = threading.Thread(
                    target=scroll_and_collect_images,
                    args=(search_variant, i, keyword_dir, progress)
                )
                threads.append(thread)
                thread.start()
            
            for thread in threads:
                thread.join()
            
            print(f"Completed crawl for search variant: {search_variant}")

    print("모든 크롤링이 완료되었습니다.")

if __name__ == "__main__":
    SAVE_DIR = "pinterest_images"
    if not os.path.exists(SAVE_DIR):
        os.makedirs(SAVE_DIR)

    try:
        start_crawling()
    except ModuleNotFoundError as e:
        print("Module 'webdriver_manager' not found. Please install it using 'pip install webdriver-manager'.")
    except Exception as e:
        print(f"An error occurred: {e}")

Starting crawl for keyword: 루피
Processing search variant: 루피


python(65714) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(65715) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(65716) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(65717) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(65718) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(65719) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(65720) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(65721) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(65722) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(65723) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(65724) Malloc

Thread 2: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Thread 0: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Thread 3: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Thread 4: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Thread 1: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Total: 100/20000 images downloaded.
Progress: 0.50% completed.
Total: 200/20000 images downloaded.
Progress: 1.00% completed.
Total: 300/20000 images downloaded.
Progress: 1.50% completed.
Total: 400/20000 images downloaded.
Progress: 2.00% completed.
Total: 500/20000 images downloaded.
Progress: 2.50% completed.
Total: 600/20000 images downloaded.
Progress: 3.00% completed.
Total: 700/20000 images downloaded.
Progress: 3.50% completed.
Total: 800/20000 images downloaded.
Progress: 4.00% completed.
Total: 900/20000 images downloaded.
Progress: 4.50% completed.
Total: 1000/20000 images downloaded.
Progress: 5.00% completed.
Total: 1100/20000 images downloaded.
Progress: 5.50% completed.
Total: 1200/20000 images downloaded.
Progress: 6.00% completed.
Total: 1300/20000 images downloaded.
Progress: 6.50% completed.
To

python(66246) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(66247) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(66248) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(66249) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(66250) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(66251) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(66252) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(66253) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(66254) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(66255) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(66256) Malloc

Thread 4: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Thread 3: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Thread 2: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Thread 0: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Thread 1: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Total: 100/20000 images downloaded.
Progress: 0.50% completed.
Total: 200/20000 images downloaded.
Progress: 1.00% completed.
Total: 300/20000 images downloaded.
Progress: 1.50% completed.
Total: 400/20000 images downloaded.
Progress: 2.00% completed.
Total: 500/20000 images downloaded.
Progress: 2.50% completed.
Total: 600/20000 images downloaded.
Progress: 3.00% completed.
Total: 700/20000 images downloaded.
Progress: 3.50% completed.
Total: 800/20000 images downloaded.
Progress: 4.00% completed.
Total: 900/20000 images downloaded.
Progress: 4.50% completed.
Total: 1000/20000 images downloaded.
Progress: 5.00% completed.
Total: 1100/20000 images downloaded.
Progress: 5.50% completed.
Total: 1200/20000 images downloaded.
Progress: 6.00% completed.
Completed crawl for search variant: 루피 밈
Processing search variant

...

Thread 4: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Thread 0: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Thread 2: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Thread 1: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Total: 100/20000 images downloaded.
Progress: 0.50% completed.
Thread 3: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Total: 200/20000 images downloaded.
Progress: 1.00% completed.
Total: 300/20000 images downloaded.
Progress: 1.50% completed.
Total: 400/20000 images downloaded.
Progress: 2.00% completed.
Total: 500/20000 images downloaded.
Progress: 2.50% completed.
Total: 600/20000 images downloaded.
Progress: 3.00% completed.
Total: 700/20000 images downloaded.
Progress: 3.50% completed.
Total: 800/20000 images downloaded.
Progress: 4.00% completed.
Total: 900/20000 images downloaded.
Progress: 4.50% completed.
Total: 1000/20000 images downloaded.
Progress: 5.00% completed.
Total: 1100/20000 images downloaded.
Progress: 5.50% completed.
Total: 1200/20000 images downloaded.
Progress: 6.00% completed.
Total: 1300/20000 images downloaded.
Progress: 6.50% completed.
To

python(74753) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(74754) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(74755) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(74756) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(74757) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(74758) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(74759) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(74760) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(74761) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(74762) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(74763) Malloc

Thread 3: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Thread 2: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Thread 1: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Thread 4: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Thread 0: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Total: 100/20000 images downloaded.
Progress: 0.50% completed.
Total: 200/20000 images downloaded.
Progress: 1.00% completed.
Total: 300/20000 images downloaded.
Progress: 1.50% completed.
Total: 400/20000 images downloaded.
Progress: 2.00% completed.
Total: 500/20000 images downloaded.
Progress: 2.50% completed.
Total: 600/20000 images downloaded.
Progress: 3.00% completed.
Total: 700/20000 images downloaded.
Progress: 3.50% completed.
Total: 800/20000 images downloaded.
Progress: 4.00% completed.
Total: 900/20000 images downloaded.
Progress: 4.50% completed.
Total: 1000/20000 images downloaded.
Progress: 5.00% completed.
Total: 1100/20000 images downloaded.
Progress: 5.50% completed.
Completed crawl for search variant: 감자 밈
Processing search variant: 감자 짤


python(75076) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(75077) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(75078) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(75079) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(75080) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(75081) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(75082) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(75083) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(75084) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(75085) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(75086) Malloc

Thread 3: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Thread 2: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Thread 4: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Thread 0: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Thread 1: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Total: 100/20000 images downloaded.
Progress: 0.50% completed.
Total: 200/20000 images downloaded.
Progress: 1.00% completed.
Total: 300/20000 images downloaded.
Progress: 1.50% completed.
Total: 400/20000 images downloaded.
Progress: 2.00% completed.
Total: 500/20000 images downloaded.
Progress: 2.50% completed.
Total: 600/20000 images downloaded.
Progress: 3.00% completed.
Total: 700/20000 images downloaded.
Progress: 3.50% completed.
Total: 800/20000 images downloaded.
Progress: 4.00% completed.
Total: 900/20000 images downloaded.
Progress: 4.50% completed.
Total: 1000/20000 images downloaded.
Progress: 5.00% completed.
Total: 1100/20000 images downloaded.
Progress: 5.50% completed.
Total: 1200/20000 images downloaded.
Progress: 6.00% completed.
Total: 1300/20000 images downloaded.
Progress: 6.50% completed.
To